In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier

import tensorflow as tf
from tensorflow.keras import layers, regularizers, initializers, Input, Model

# Data set and preparation

We use the same MNIST data set for handwritten digit recognition as in session #02. The scikit-learn loader function will automatically download the data set and cache it on the local disk.

In [ ]:
mnist = fetch_openml("mnist_784")
mnist.data.shape

As before, we visualise the $28\times 28$ images and the corresponding weight matrices by combining them into an image map.

In [ ]:
def mk_imagemap(data, nrow, ncol, padding=2):
    w, h = data.shape[-2:]
    data = data.reshape((-1, w, h))
    n = data.shape[0]
    image = np.zeros((nrow * h + (nrow - 1) * padding, ncol * w + (ncol - 1) * padding))
    y = 0
    k = 0
    for i in range(nrow):
        x = 0
        for j in range(ncol):
            if k <= n - 1:
                image[y:y+h, x:x+w] = data[k]
            x += w + padding
            k += 1
        y += h + padding
    return image

plt.imshow(mk_imagemap(mnist.data.to_numpy().reshape((7000, 10, 28, 28)), 5, 10), cmap="binary");

Rescale data from range $0\ldots 255$ to a more reasonable scale $[0, 1]$. The unusual range of the original features tends to throw the regularisation of some machine learning algorithms off balance (which then need very unusual regularisation parameters to work well). We also convert the target categories to numeric codes $0, \ldots, 9$ (as expected by deep learning frameworks).

In [ ]:
X = mnist.data.to_numpy() / 255
y = mnist.target.to_numpy().astype('int')

Split data set into 20% test data and 80% training data.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print(X_train.shape)
print(X_test.shape)

# Baseline: traditional machine learning

As a baseline, we repeat the experiments with standard machine learning classifiers, comparing linear and non-linear models. Linear classifiers have the advantage that their parameter matrices can be interpreted as pixel weights.

Let's start with a standard linear classifier. SVMs are traditionally used as off-the-shelf models because they tend to be robust without too much metaparameter optimisation. However, training takes its time even with the optimised LibLinear, so you might want to skip this cell for now.

In [ ]:
%%time
clf = LinearSVC(C=1, max_iter=1000)
clf.fit(X_train, y_train)
print('Training accuracy: {:.5f}'.format(clf.score(X_train, y_train)))
result = clf.predict(X_test)
print(classification_report(y_test, result, digits=4))

Stochastic gradient descent is much faster with multicore processing, but very sensitive to regularisation parameter.  With the right choice of $\alpha$, we can achieve $> 90\%$ accuracy in just a few seconds of training time.

In [ ]:
%%time
clf = SGDClassifier(alpha=1e-4, max_iter=5000, n_jobs=-1)
clf.fit(X_train, y_train)
print('Training accuracy: {:.5f}'.format(clf.score(X_train, y_train)))
result = clf.predict(X_test)
print(classification_report(y_test, result, digits=4))

An important advantage of linear classifiers for this exercise is that we can easily visualise the feature weights learned by the classifier. Here, we can roughly make out the shapes of the corresponding digits in the plot (or at least some of them).

In [ ]:
def plot_weights(W, nrow=2, ncol=5, size=28, cmap='bwr', vmax=None):
    image = mk_imagemap(W.reshape((-1, size, size)), nrow, ncol)
    if vmax is None:
        vmax = np.abs(image).max()
        print(f"range: [{-vmax:.2f}, {vmax:.2f}]")
    plt.imshow(image, cmap=cmap, vmin=-vmax, vmax=vmax)

plot_weights(clf.coef_, vmax=1.5)

The classification accuracy is still unsatisfactory, though, and there is no evidence of overtraining, indicating that simple linear classifiers are not flexible enough. An SVM with _rbf_ kernel performs much better in fact, but takes very long to train. Instead, let us try a **nearest neighbours** classifier, which should work quite well given the large amount of training data for each class and the fact that – while there are different “writing styles” for some digits – many exemplars tend to look highly similar.  The result indeed shows a substantial improvement over SGD.

In [ ]:
%%time
nn = KNeighborsClassifier(n_neighbors=10, metric='manhattan')
nn.fit(X_train, y_train)
result = nn.predict(X_test)
print(classification_report(y_test, result, digits=4))

# Neural Networks with TensorFlow

For some of our deep learning models, the feature vectors need to be reshaped into $28\times 28$ matrices, so that we can exploit the two-dimensional structure of the images with convolutional layers later on.  The target categories should in principle be converted to one-hot encodings, but TensorFlow accepts integer class codes with sparse categorical cross-entropy loss.

In [ ]:
X_train = X_train.reshape(-1, 28, 28)
X_test = X_test.reshape(-1, 28, 28)

In [ ]:
X_train.shape

Neural networks are defined in TensorFlow by creating layer objects and connecting them via function application, feeding the output of one layer as input argument to the next layer. Our first neural network consists of single fully connected layer with softmax activation function and corresponds to a logistic regression linear classifier.

In [ ]:
inputs = Input(shape=(28, 28))
flatten = layers.Flatten() # convert 28x28 image back to feature vector
layer1 = layers.Dense(10, activation='softmax', # extra arguments below are useful for visualisation
                      kernel_initializer='zeros', kernel_regularizer=regularizers.L1(l1=1e-6))
flatten_out = flatten(inputs)
layer1_out = layer1(flatten_out)
model = Model(inputs=inputs, outputs=layer1_out)

Unlike Scikit-Learn, a neural network model has to be compiled (into a compute graph that can be executed e.g. on a GPU) before it can be trained. In this step, we also specify the loss function (remember that we need _sparse categorical cross-entropy_ for our training data), the parameter optimisation algorithm (here: adaptive SGD with momentum), and the evaluation metric (here: classification accuracy).

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Now we can train the neural network. Here, we train for a total of 20 epochs, using 256 images in each SGD batch. The `verbose` parameter specifies how much progress information is displayed during training. The `validation_data` parameter allows us to monitor the accuracy on a validation set as an indication of over-training (normally, we should not use the test set but a separate validation set for this purpose, of course).

In [ ]:
res = model.fit(X_train, y_train, epochs=30, batch_size=512, verbose=2,
                validation_data=(X_test, y_test))

The final model can easily be evaluated on the test data:

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {100 * test_acc:.3f}%')

The predictions made by the model are actually probability distributions over the 10 categories. In order to predict the integer values (category codes), we need to find the most probable category for each item.

In [ ]:
predictions = model.predict(X_test, verbose=0)
print(np.round(predictions[:10, :], 3))
predictions = np.argmax(predictions, axis=1)
print(predictions[:10])

Verify the evaluation score above using Scikit-Learn evaluation metrics and obtain P/R/F for each digit.

In [ ]:
print('Accuracy:', accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions, digits=4))

Training the neural network with the `fit()` method has returned an object `res`, which we can use (amongst other things) to plot learning curves across the training epochs. **NB:** These are not the learning curves we're usually interested in, which show how the amount of training data affects the final accurac (while the training process shown here always uses the full data set).  It is much more time-consuming to produce such “true” learning curves because the training has to be restarted from scratch for each data point to be plotted.

In [ ]:
plt.plot(res.history['accuracy'], linewidth=3)
plt.plot(res.history['val_accuracy'], linewidth=3)
plt.axis(ymin=.8, ymax=1)
plt.xlabel("# epochs"); plt.ylabel("accuracy")
plt.legend(("training data", "test data"));

As with linear models in Scikit-Learn, we can read out the feature weights and bias terms for each category and visualise them.  Note that the `layer1` object has been integrated into `model`, but it still encapsulates the trained parameters for this layer.  We could also retrieve weights from `model.layers[2]`, but it's too easy to confuse different layers that way.

In [ ]:
W = layer1.get_weights()[0]
print(W.shape)
b = layer1.get_weights()[1]
print(b)

For visualisation of the learned weights, we need to transpose the weights matrix into the usual $10\times 784$ format.

In [ ]:
plot_weights(W.T)

> **Exercise:** Re-train the neural network with different meta-parameter settings and different numbers of epochs and check how this affects (i) final accuracy, (ii) learning curves and overtraining, and (iii) the feature weights learned by the network (using the visualisation above).

# Deep Learning

A singe-layer neural network is still a linear classifier (in our case equivalent to a multinomial logistic regression model), so it cannot account for multiple prototypes representing different styles of writing a digit and fails to achieve much better accuracy than a linear SVM or logistic regression with SGD. However, we can much more easily change the neural network to a non-linear classifier by adding a “hidden” layer, which must also have a non-linear activation function.  Even such a two-layer network can learn very complex decision boundaries, depending on the number of neurons in the hidden layer.

Intuitively, we want the hidden layer to represent distinct styles for each digit (which should easily be learned by a linear classifier), i.e. each neuron should detect one particular digit shape. The second and final layer only has to learn how to map the shapes to digits.  Some digits appear to be fine with a single prototype (e.g. 0 and 3, whose shapes are easily recognised in the visualisation of the weights matrix), while others with very blurry shapes (e.g. 1, 4, 6, 9) may need more than three prototypes.  This intuitive reasoning would suggest between 20 and 30 neurons for the hidden layer, but some experimentation shows that the network learns much better with a larger hidden layer.  Training also takes more epochs than before and we need to allow for some overtraining to achieve optimal test accuracy.

The metaparameter settings below are rather uncommon and have been selected to bring out a reasonable visualisation.

In [ ]:
%%time

inputs = Input(shape=(28, 28))
flatten = layers.Flatten()
hidden = layers.Dense(48, activation='sigmoid', kernel_regularizer=regularizers.L1(l1=1e-5))
final = layers.Dense(10, activation='softmax', kernel_regularizer=regularizers.L1(l1=1e-4))
hidden_out = hidden(flatten(inputs))
final_out = final(hidden_out)

model = Model(inputs=inputs, outputs=final_out)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# set verbose=1 or verbose=2 below to follow training progress
res = model.fit(X_train, y_train, epochs=50, batch_size=256, verbose=0,
                validation_data=(X_test, y_test))

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {100 * test_acc:.3f}%')

This neural network already outperforms the nearest neighbours classifier!  However, a look at the weights matrix of the output layer (in our displays, rows correspond to output neurons and columns to the first 16 neurons of the hidden layer) shows that our intuition didn't quite work out.  We expected that each neuron of the hidden layer might learn to recognise one particular digit style, so it would ideally map only to the corresponding output neuron (i.e. each column of the matrix displayed belowed should be close to a one-hot vector).  But in fact, most hidden layer neurons contribute to many output neurons with positive _and_ negative weights.

In [ ]:
W2 = final.get_weights()[0]
print(np.round(W2, 1))

As a result, the feature weights of the hidden layer aren't immediately recognisable as digit styles, though some of the shapes are reminiscent of (parts of) digits, often mixed together in various combinations.

In [ ]:
W1 = hidden.get_weights()[0].T.reshape(-1, 28, 28)
plot_weights(W1, 6, 8, vmax=1.5)

> **Exercise:** Experiment with different regularisations, layer initialisations and network designs.  Which parameter settings improve classification accuracy?  Do the same settings also result in more recognisable feature weights in the visualisation?  What happen if you increase or decrease the number of neurons in the hidden layer?
> 
> **Exercise:** Experiment with deeper network topologies.  One of the fundamental insights of Deep Learning is that having more layers leads to better and more robust learning results than having more neurons in a single hidden layer.  Is this also the case for handwritten digit recognition, or is the task too simple?

As we start to experiment more, we will want to re-use code by encapsulating repetitive tasks in subroutines.  We will still have to create the neural networks from scratch as we want to change their topologies in the experiments, but compiling the model, training and evaluation are always the same, so we can wrap them in a support function.

In [ ]:
def train_and_eval(inputs, outputs, epochs=42, batch_size=256, verbose=1, plot=False):
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    res = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose,
                    validation_data=(X_test, y_test))

    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f'Test accuracy: {100 * test_acc:.3f}%')

    if plot:
        plt.plot(res.history['accuracy'], linewidth=3)
        plt.plot(res.history['val_accuracy'], linewidth=3)
        plt.axis(ymin=.8, ymax=1)
        plt.xlabel("# epochs"); plt.ylabel("accuracy")
        plt.legend(("training data", "test data"))

    return model

As an example, here is a neural network with two fairly large hidden layers. Without regularisation, the network learns very quickly and overtrains substantially, but it still achieves better results on the test data.

In [ ]:
%%time

inputs = Input(shape=(28, 28))
flatten = layers.Flatten()
hidden1 = layers.Dense(128, activation='sigmoid')
hidden2 = layers.Dense(64, activation='sigmoid')
final = layers.Dense(10, activation='softmax')
final_out = final(hidden2(hidden1(flatten(inputs))))

model = train_and_eval(inputs, final_out, epochs=42, verbose=2, plot=True)

It's hard to make much sense of the image maps learned by the first layer, though.

In [ ]:
W1 = hidden1.get_weights()[0].T.reshape(-1, 28, 28)
plot_weights(W1, 8, 16, vmax=1.2)

# Convolutional neural networks (CNN)

For image analysis, a huge disadvantage of simple fully-connected layers is that they learn feature weights for each individual pixel of an image rather than to recognise shapes that can appear in different positions.  This is achieved by convolutional layers, which have become a standard approach for image classification tasks.  Here, we try a simple approach with a single convolutional hidden layer, which learns “filters” of size $15\times 15$ pixels that are swept across the image. A pooling layer reduces these to a lower resolution ($7\times 7$ positions for the 64 filter channels), which are then fed into a dense final layer.

**NB:** Convolutional layers are designed for image analysis and assume that there are multiple colour channels, so for our greyscale image the input is a $28\times 28\times 1$ tensor rather than a $28\times 28$ matrix.  Fortunately, the input layer can make this adjustment automatically if we specify the right shape.

In [ ]:
inputs = Input(shape=(28, 28, 1))
cnn = layers.Conv2D(64, (15, 15), activation='relu')
z = cnn(inputs) # convolutional layer with 64 filters of 15x15 pixels
z = layers.MaxPooling2D((2,2))(z) # reduce resolution by taking maximum over each 2x2 square of filter positions
z = layers.Flatten()(z) # flatten the 7x7x1 tensor for final dense layer
outputs = layers.Dense(10, activation='softmax')(z)

# Compile the model
model = train_and_eval(inputs, outputs, epochs=20, plot=True)

Printing a summary can help us understand what happens in the different layers of the model.

In [ ]:
model.summary()

A visualisation of the convolutional filters suggests that they recognise certain shapes, which could be part of digits, but it's difficult to make them out very clearly.  **Exercise:** perhaps you can find a CNN topology that generates more visually appealing features?

In [ ]:
W1 = cnn.get_weights()[0].transpose((2, 3, 0, 1))[0] # weights tensor has shape (15, 15, 1, 64)
plot_weights(W1, 8, 8, size=15, vmax=.25)

> **Exercise:** Can you think of other visualisations that could give us insights into what the CNN has learned?

As a last step, let's try a deep network with multiple CNN and dense layers.  It takes a lot of experience to guess which network topologies work best, where to add layers, etc.  You will also need to add some form of regularisation in order to keep very deep networks from overtraining drastically.  A very simple strategy that can work surprisingly well is to simply stop training early, e.g. when validation loss stops decreasing or even starts to increase again.  More advanced strategies will be explored in further sessions of this course.

In [ ]:
inputs = Input(shape=(28, 28, 1))
cnn1 = layers.Conv2D(32, (5, 5), activation='relu')
z = cnn1(inputs) # first CNN layer learns small shape fragments of 5x5 pixels
cnn2 = layers.Conv2D(64, (9, 9), activation='relu')
z = cnn2(z)      # second CNN layer then should recognise larger building blocks
z = layers.MaxPooling2D((4, 4))(z) # reduce resolution of these blocks for final dense layers
z = layers.Flatten()(z) # flatten the 4x4x1
hidden1 = layers.Dense(64, activation='sigmoid')
z = hidden1(z)   # hidden layer over the feature map
final = layers.Dense(10, activation='softmax')
outputs = final(z) # final classification layer

# Compile the model
model = train_and_eval(inputs, outputs, epochs=20, verbose=2, plot=True)

Note how many parameters this model has (more than 230,000), so it should be very prone to overtraining.  Due to the deep topology of the network, we still obtain very good classification accuracy on the test set.

In [ ]:
model.summary()

> **Exercise:** Can you further improve on this model? Does it help to add even more layers? Try reducing overtraining by adding layer regularisation or with the help of `Dropout` layers.